In [ ]:
import os
import json
import string
from pprint import pprint
from pathlib import Path
from collections import defaultdict

from dotenv import load_dotenv
from langchain.llms import OpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory


load_dotenv()

## Scenarios

1. Job Interview
2. Apology and Forgiveness
3. Parent-Child Interaction

### Data Structure

``` json
{
    "theme": "job-interview",
    "actors": [{"name": "Mr. John", "role": "interviewer"}, 
               {"name": "Jane", "role": "interviewee"}],
    "conversation": [{ "call": "Hallo Jane, Wie geht's dir heute", 
                       "response": "I am fine, thank you.", 
                        "target": "Es geht mir gut, danke." }]
}

```

## Utils

In [ ]:
def load_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


def get_roleplay_data(roleplay_path: str) -> list:
    """Get Roleplay Paths """
    roleplay_paths = [i for i in os.listdir("roleplay_data") if i.endswith(".json")]
    roleplay_data = [load_json_file(os.path.join(roleplay_path, i)) for i in roleplay_paths]
    return roleplay_data

In [ ]:
roleplay_data = get_roleplay_data("roleplay_data")

## The Tutor

In [ ]:
from dotenv import load_dotenv
from langchain.llms import OpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.cache import SQLiteCache
import langchain


#langchain.llm_cache = SQLiteCache(database_path=".langchain.db")
load_dotenv()

In [ ]:
def initialiaze_tutor(temperature: float=0.5):
    prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(f"""
        You are a German tutor that tries to help you student provide the correct German translation for an English statement.
        For each of your student's responses provide constructive feedbacks and Hints to guide them to the right answer. 
        Applaud them when they finally get the right translation
        """                               
    ),
    AIMessagePromptTemplate.from_template(f"Could you provide the German translation for this English sentence: {response_target}"),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
    ])


    llm = ChatOpenAI(temperature=temperature)
    memory = ConversationBufferMemory(return_messages=True)
    conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm, verbose=True)
    return memory, conversation

def execute_convo(conversation, input):
    output = conversation.run(input=input)
    print(output)

In [ ]:
call, call_target, response, response_target = roleplay_data[0]["conversation"][0].values()
memory, conversation = initialiaze_tutor(0.4)
initial_input = f""" 
Challenge:
How would you say this English statement in German: {response_target}
target answer: {response}. Remember not to provide the right answer until the student gets the answer.
Provide helpful hints for the student. Avoid providing any translations in your hints.
Give the student a maximum of three attempts before revealing the final answer.
List my instructions and confirm your understanding of them
"""

In [ ]:
execute_convo(conversation, initial_input)

In [ ]:
execute_convo(conversation, "Guten Morgen! Danke. Ich habe drei jahre ProjektManagement Erpharung und Team Leitung")

In [ ]:
execute_convo(conversation, "Guten Morgen! Danke. Ich habe drei jahre Projektmanagement Erfahrung und Team Leitung")

In [ ]:
execute_convo(conversation, "Guten Morgen! Danke. Ich habe drei Jahre Projektmanagement Erfahrung und Team Leitung")

In [ ]:
conversation.run(input="Guten Morgen! Danke. Ich habe drei jahre ProjektManagement Erpharung und Team Leitung")

In [ ]:
conversation.run(input="Guten Morgen! Danke. Ich habe drei jahre Projektmanagement Erfahrung und Teamführung")